# Bootstrap Internal Validation

In this notebook, we will describe the code that implements the procedure we use to perform bootstrap-based internal validation of the models compared in our paper. We show the application of this validation procedure to the conventional parameter model. A similar procedure was applied to the deep learning model. 

In order to get a sense of how well our model would generalize to an external validation cohort, we assessed its predictive accuracy within the training sample using a bootstrap-based procedure recommended in the guidelines for *Transparent Reporting of a multivariable model for Individual Prognosis Or Diagnosis (Tripod)*. This procedure attempts to derive realistic, 'optimism-adjusted' estimates of the model's generalization accuracy using the training sample

We first import required libraries:

In [ ]:
import os, sys, pickle
import optunity, lifelines
from lifelines.utils import concordance_index
import numpy as np

Next, we import the functions required to train the conventional parameter model:

In [ ]:
sys.path.insert(0, '../code')
from CoxReg_Single_run import *
from hypersearch import *

Import data, where `x_full` is the $n \times p$ input matrix of volumetric measures ($n$=sample size, $p$=dimensionality of input vector), `y_full` is an $n \times 2$ matrix of outcomes where column 1 represents censoring status and column 2 represents survival/censoring time. 

In [ ]:
with open('../data/inputdata_conv.pkl', 'rb') as f: c3 = pickle.load(f)
x_full = c3[0]
y_full = c3[1]
del c3

Initialize empty lists to store predictions and performance measures:

In [ ]:
preds_bootfull = []
inds_inbag = []
Cb_opts  = []

Now we implement each step of the bootstrap internal validation procedure, as outlined in the manuscript:

### Step 1
Train a prediction model on the full sample:
#### 1(a) : Find optimal hyperparameters

In [ ]:
opars, osummary = hypersearch_cox(x_data=x_full, y_data=y_full, method='particle swarm', nfolds=6, nevals=50, penalty_range=[-2,1])

#### 1(b) : using optimal hyperparameters, train a model on full sample

In [ ]:
omod = coxreg_single_run(xtr=x_full, ytr=y_full, penalty=10**opars['penalty'])

#### 1(c) : Compute Harrell's Concordance index ($C_{full}^{full}$)

In [ ]:
predfull = omod.predict_partial_hazard(x_full)
C_app = concordance_index(y_full[:,1], -predfull, y_full[:,0])

print('\n\n==================================================')
print('Apparent concordance index = {0:.4f}'.format(C_app))
print('==================================================\n\n')

`C_app` ($C_{full}^{full}$) represents the apparent predictive accuracy, i.e. the inflated accuracy obtained when a model is tested on the same sample on which it was trained/optimized 

In the next steps, we use bootstrap sampling to estimate the optimism, which we then use to adjust the apparent predictive accuracy.

#### Bootstrap sampling
We will take B = 100 bootstrap samples

In [ ]:
#define useful variables
nsmp = len(x_full)
rowids = [_ for _ in range(nsmp)]
B = 100

for b in range(B):
    print('\n-------------------------------------')
    print('Current bootstrap sample:', b, 'of', B-1)
    print('-------------------------------------')

    #STEP 2: Generate a bootstrap sample by doing n random selections with replacement (where n is the sample size)
    b_inds = np.random.choice(rowids, size=nsmp, replace=True)
    xboot = x_full[b_inds]
    yboot = y_full[b_inds]

    #(2a) find optimal hyperparameters
    bpars, bsummary = hypersearch_cox(x_data=xboot, y_data=yboot, method='particle swarm', nfolds=6, nevals=50, penalty_range=[-2,1])
    
    #(2b) using optimal hyperparameters, train a model on bootstrap sample
    bmod = coxreg_single_run(xtr=xboot, ytr=yboot, penalty=10**bpars['penalty'])
    
    #(2c[i])  Using bootstrap-trained model, compute predictions on bootstrap sample. Evaluate accuracy of predictions (Harrell's Concordance index)
    predboot = bmod.predict_partial_hazard(xboot)
    Cb_boot = concordance_index(yboot[:,1], -predboot, yboot[:,0])
    
    #(2c[ii]) Using bootstrap-trained model, compute predictions on FULL sample.     Evaluate accuracy of predictions (Harrell's Concordance index)
    predbootfull = bmod.predict_partial_hazard(x_full)
    Cb_full = concordance_index(y_full[:,1], -predbootfull, y_full[:,0])

    #STEP 3: Compute optimism for bth bootstrap sample, as difference between results from 2c[i] and 2c[ii]
    Cb_opt = Cb_boot - Cb_full
    
    #store data on current bootstrap sample (predictions, C-indices)
    preds_bootfull.append(predbootfull)
    inds_inbag.append(b_inds)
    Cb_opts.append(Cb_opt)

    del bpars, bmod

Now we compute bootstrap-estimated optimism, by averaging the optimism estimates across the B bootstrap samples: $$\frac{1}{B}\sum_{b=1}^{B} \bigg( C_{b}^{b} - C_{b}^{full} \bigg)$$

In [ ]:
C_opt = np.mean(Cb_opts)

Now we adjust the apparent C using the bootstrap-estimated optimism:

In [ ]:
C_adj = C_app - C_opt

Next, we compute confidence intervals for optimism-adjusted C:

In [ ]:
C_opt_95confint = np.percentile([C_app - o for o in Cb_opts], q=[2.5, 97.5])

In [ ]:
print('Optimism bootstrap estimate = {0:.4f}'.format(C_opt))
print('Optimism-adjusted concordance index = {0:.4f}, and 95% CI = {1}'.format(C_adj, C_opt_95confint))